# Feature Engineering Part 1

On this Noteboook, the process of Feature Enginnering and Feature Selection is debugged using reduced data. After the definition of the process, all the code generated is applied to the complete data on Part 2.

## Libraries

In [22]:
import numpy as np
import pandas as pd
from cnr_methods import get_simplified_data, transform_data

# Feature Engineering Library for Time Series
from tsfresh import extract_features
from tsfresh.utilities.dataframe_functions import make_forecasting_frame
from tsfresh.utilities.dataframe_functions import impute

# Feature Selection Libraries
from cnr_methods import LOFO_GPU_Importance
import xgboost as xgb

#Ignore Warnings
import warnings
warnings.filterwarnings('ignore')

## Read Data

For this pipeline, only Training Set will be used.

In [23]:
full_data, y_train = get_simplified_data()
train_data = full_data[full_data['Set']=='Train']

In [24]:
train_data.head()

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.44,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.26,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.707402,-0.853745,0.997093,-1.415138,287.00,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.78,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.46,95.905879,Train


To simplify the work, we will generate features for just one Wind Farm. When doing modelling, the features, as the models, will be generated for all Wind Farms separately.

In [25]:
WF = 'WF1'
data = train_data[train_data['WF']==WF]
y_train = y_train[y_train.index.isin(data['ID'])]

In [26]:
data

,ID,WF,U_100m,V_100m,U_10m,V_10m,T,CLCT,Set
Time,,,,,,,,,
2018-05-01 01:00:00,1,WF1,-2.248500,-3.257800,1.254603,-0.289687,286.440000,82.543144,Train
2018-05-01 02:00:00,2,WF1,-2.434500,-1.446100,2.490908,-0.413370,286.260000,99.990844,Train
2018-05-01 03:00:00,3,WF1,-1.707402,-0.853745,0.997093,-1.415138,287.000000,98.367235,Train
2018-05-01 04:00:00,4,WF1,3.706500,-6.217400,0.689598,-0.961441,284.780000,94.860604,Train
2018-05-01 05:00:00,5,WF1,3.813400,-5.444600,0.290994,-0.294963,284.460000,95.905879,Train
...,...,...,...,...,...,...,...,...,...
2019-01-15 20:00:00,6235,WF1,-2.211200,-7.284650,-0.075525,-2.365039,280.085000,0.000000,Train
2019-01-15 21:00:00,6236,WF1,-1.414258,-5.488200,-0.306579,-2.134722,280.312534,0.000000,Train
2019-01-15 22:00:00,6237,WF1,-2.825250,-5.779000,-0.574004,-1.896251,279.285000,0.000000,Train


## Feature Creation

First, using the Zonal and Meridional Components of Wind, the Magnitude and Direction of Wind Vector for 100m and 10m height.

### Wind Speed Vector

In [27]:
feature_data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]
feature_data['Wind Speed 100m'] = np.sqrt(feature_data['U_100m']**2 + feature_data['V_100m']**2)
feature_data['Wind Direction 100m'] = np.arctan(feature_data['V_100m']/feature_data['U_100m'])
feature_data['Wind Speed 10m'] = np.sqrt(feature_data['U_10m']**2 + feature_data['V_10m']**2)
feature_data['Wind Direction 10m'] = np.arctan(feature_data['V_10m']/feature_data['U_10m'])

Changing Reference for Negative Angles:

In [28]:
feature_data['Wind Direction 100m'] = feature_data['Wind Direction 100m'].apply(lambda x: 360 + x if x < 0 else x)
feature_data['Wind Direction 10m'] = feature_data['Wind Direction 10m'].apply(lambda x: 360 + x if x < 0 else x)

Using the original Features, we will create some variables over the Numerical Variables from the simplified data.

In [29]:
features = ['T', 'CLCT', 'U_100m','V_100m','U_10m','V_10m']

### Time-Relative Variables

Here,  Values for Last Week and Month for each Numerical Feature are generated.

In [30]:
for column in features:
    feature_data[column + '_last_week'] = feature_data[column].shift(7)
    feature_data[column + '_last_month'] = feature_data[column].shift(30)

Now, Month and Quarter Statistics(Mean,Median,Variance) are generated:

In [31]:
feature_data['Month_Number'] = feature_data.index.month
feature_data['Quarter_Number'] = feature_data.index.quarter

In [32]:
# Month
mean_month = feature_data.groupby('Month_Number').mean()[features]
median_month = feature_data.groupby('Month_Number').median()[features]
variance_month = feature_data.groupby('Month_Number').var()[features]

# Quarter
mean_quarter = feature_data.groupby('Quarter_Number').mean()[features]
median_quarter = feature_data.groupby('Quarter_Number').median()[features]
variance_quarter = feature_data.groupby('Quarter_Number').var()[features]

In [33]:
# Month
mean_month.columns = mean_month.columns + '_Month_Mean'
median_month.columns = median_month.columns + '_Month_Median'
variance_month.columns = variance_month.columns + '_Month_Variance'

# Quarter
mean_quarter.columns = mean_quarter.columns + '_Quarter_Mean'
median_quarter.columns = median_quarter.columns + '_Quarterh_Median'
variance_quarter.columns = variance_quarter.columns + '_Quarter_Variance'

In [34]:
# Month
feature_data = feature_data.merge(mean_month,on='Month_Number',how='left')
feature_data = feature_data.merge(median_month,on='Month_Number',how='left')
feature_data = feature_data.merge(variance_month,on='Month_Number',how='left')

# Quarter
feature_data = feature_data.merge(mean_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(median_quarter,on='Quarter_Number',how='left')
feature_data = feature_data.merge(variance_quarter,on='Quarter_Number',how='left')

In [35]:
feature_data.index = data.index

For periodical Features, here represented by days (Of Month, Week and Year), hour and minutes, the features are applied to sinusoidal functions to replicate the cyclic nature of the variables.

In [36]:
day = feature_data.index.day
hour = feature_data.index.hour
minute = feature_data.index.minute
dayofweek = feature_data.index.dayofweek
dayofyear = feature_data.index.dayofyear

In [37]:
days_in_month = feature_data.index.days_in_month

In [38]:
feature_data["cos_day"], feature_data["sin_day"] = (
    np.cos(2 * np.pi * (day - 1) / days_in_month),
    np.sin(2 * np.pi * (day - 1) / days_in_month),
    )

feature_data["cos_hour"], feature_data["sin_hour"] = (
    np.cos(2 * np.pi * hour / 24),
    np.sin(2 * np.pi * hour / 24),
    )

feature_data["cos_minute"], feature_data["sin_minute"] = (
    np.cos(2 * np.pi * minute / 60),
    np.sin(2 * np.pi * minute / 60),
)

feature_data["cos_dayofyear"], feature_data["sin_dayofyear"] = (
    np.cos(2 * np.pi * (dayofyear - 1) / 365),
    np.sin(2 * np.pi * (dayofyear - 1) / 365),
)

feature_data["cos_dayofweek"], feature_data["sin_dayofweek"] = (
    np.cos(2 * np.pi * dayofweek / 7),
    np.sin(2 * np.pi * dayofweek / 7),
)

### Distance from Features

Distance of Position of Max and Min (Already on Tsfresh, check it later):

In [39]:
for column in features:
    feature_data[column + '_Distance_Max'] = feature_data.index - feature_data[column].idxmax()
    feature_data[column + '_Distance_Min'] = feature_data.index - feature_data[column].idxmin()
    feature_data[column + '_Distance_Max'] = feature_data[column + '_Distance_Max'].apply(lambda x : x.days)
    feature_data[column + '_Distance_Min'] = feature_data[column + '_Distance_Min'].apply(lambda x : x.days)

Dropping Original Features:

In [40]:
feature_data = feature_data.drop(features,axis=1)

## Tsfresh

Now we use Tsfresh, a Python Library that automates Feature Engineering for Time Series Data. We generate new features for all the columns on the Simplified Data, as done below.

A list of the generated features can be found https://tsfresh.readthedocs.io/en/latest/text/list_of_features.html.

In [41]:
data = data[['ID','WF','U_100m','V_100m','U_10m','V_10m','T','CLCT','Set']]

In [21]:
tsfresh_data = pd.DataFrame()
for variable in ['U_100m','V_100m','U_10m','V_10m','T','CLCT']: 
    df_shift, y = make_forecasting_frame(data[variable],kind=variable,max_timeshift=20,rolling_direction=1)
    X = extract_features(df_shift, column_id="id", column_sort="time", column_value="value", impute_function=impute,show_warnings=False,n_jobs=3)
    X['Feature'] = variable
    tsfresh_data = tsfresh_data.append(X)

Feature Extraction:  93%|█████████▎| 14/15 [02:16<00:08,  8.95s/it]

KeyboardInterrupt: 

Here, a little preprocess is done to convert the generated features to the appropriate tabular format.

In [22]:
tsfresh_data = tsfresh_data.pivot(columns='Feature')

In [23]:
tsfresh_data.columns = tsfresh_data.columns.map('{0[0]}|{0[1]}'.format)

Constant Columns are removed from data.

In [24]:
tsfresh_data = tsfresh_data.loc[:, tsfresh_data.apply(pd.Series.nunique) != 1]

In [25]:
tsfresh_data.head()

,value__abs_energy|CLCT,value__abs_energy|T,value__abs_energy|U_100m,value__abs_energy|U_10m,value__abs_energy|V_100m,value__abs_energy|V_10m,value__absolute_sum_of_changes|CLCT,value__absolute_sum_of_changes|T,value__absolute_sum_of_changes|U_100m,value__absolute_sum_of_changes|U_10m,...,value__variance|U_100m,value__variance|U_10m,value__variance|V_100m,value__variance|V_10m,value__variance_larger_than_standard_deviation|CLCT,value__variance_larger_than_standard_deviation|T,value__variance_larger_than_standard_deviation|U_100m,value__variance_larger_than_standard_deviation|U_10m,value__variance_larger_than_standard_deviation|V_100m,value__variance_larger_than_standard_deviation|V_10m
id,,,,,,,,,,,,,,,,,,,,,
2018-05-01 02:00:00,6813.370572,82047.8736,5.055752,1.574030,10.613261,0.083919,0.000000,0.00,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 03:00:00,16811.539518,163992.6612,10.982542,7.778652,12.704466,0.254793,17.447701,0.18,0.186000,1.236305,...,0.008649,0.382112,0.820564,0.003824,1.0,0.0,0.0,0.0,0.0,0.0
2018-05-01 04:00:00,26487.652359,246361.6612,13.897765,8.772845,13.433347,2.257409,19.071310,0.92,0.913098,2.730120,...,0.095117,0.425138,1.045847,0.253942,1.0,0.0,0.0,0.0,1.0,0.0
2018-05-01 05:00:00,35486.186521,327461.3096,27.635908,9.248391,52.089410,3.181777,22.577941,3.14,6.327000,3.037614,...,6.458769,0.467797,4.356622,0.202684,1.0,0.0,1.0,0.0,1.0,0.0
2018-05-01 06:00:00,44684.124127,408378.8012,42.177927,9.333069,81.733079,3.268781,23.623216,3.46,6.433900,3.436218,...,8.384555,0.556415,4.485969,0.198239,1.0,0.0,1.0,0.0,1.0,0.0


## Feature Selection

Finally, all the Features Generated have to be filtered, so only the most relevant ones are passed to the model, as a way of avoiding Overfitting. For this process, a process called LOFO (Leave One Feature Out) is selected.

The process is simple: For each Feature, a arbitrary Model (Here a XGBoost) is cross validated on a dataset that contains all the features except one, which importance is being measured, and the precision without that feature is compared to a baseline, where all features are present. The features whose removal leads to worse results are considered most important features to the dataset.

More info about this process can be found on https://github.com/aerdem4/lofo-importance.

The implementation of this method on this however, was an adaptation of the original code to allow use of GPU resources as a faster way to obtain results, since this work involves a big number of columns.

In [27]:
final_features = feature_data.merge(tsfresh_data,left_on=feature_data.index,right_on=tsfresh_data.index,how='left')
final_features = final_features.rename({'key_0':'Date'},axis=1)

#final_features = feature_data

In [28]:
features = final_features.drop(['ID','WF','Set','Date'],axis=1).columns

#features = final_features.drop(['ID','WF','Set'],axis=1).columns

In [29]:
final_features = transform_data(final_features[features])
y_train = transform_data(y_train[['Production']])

In [30]:
param = {'tree_method' : 'gpu_hist'}

In [31]:
importance_df = LOFO_GPU_Importance(final_features,y_train,features,param)

8 s/it
1395/2194 12.721524 s/it
1396/2194 12.727840 s/it
1397/2194 12.795303 s/it
1398/2194 12.819725 s/it
1399/2194 12.853228 s/it
1400/2194 12.758726 s/it
1401/2194 12.789562 s/it
1402/2194 12.759610 s/it
1403/2194 12.803520 s/it
1404/2194 12.752862 s/it
1405/2194 12.806856 s/it
1406/2194 12.764817 s/it
1407/2194 12.813956 s/it
1408/2194 12.796551 s/it
1409/2194 12.793470 s/it
1410/2194 12.832509 s/it
1411/2194 12.765317 s/it
1412/2194 12.659596 s/it
1413/2194 12.844996 s/it
1414/2194 12.803408 s/it
1415/2194 12.786733 s/it
1416/2194 12.781104 s/it
1417/2194 12.801320 s/it
1418/2194 12.762280 s/it
1419/2194 12.799303 s/it
1420/2194 12.785680 s/it
1421/2194 12.853196 s/it
1422/2194 12.723443 s/it
1423/2194 12.889119 s/it
1424/2194 12.783448 s/it
1425/2194 12.752692 s/it
1426/2194 12.694665 s/it
1427/2194 12.812153 s/it
1428/2194 12.835797 s/it
1429/2194 12.921110 s/it
1430/2194 12.790737 s/it
1431/2194 12.868519 s/it
1432/2194 12.780747 s/it
1433/2194 12.881987 s/it
1434/2194 12.81048

In [32]:
importance_df.to_csv(r'Feature Selection\Importance_WF1.csv')